In [ ]:
#  建模思路
#  1. 一个商场一个场景
#  2. 把一个场景看成多分类问题

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc

In [2]:
DATA_DIR = "./data/"
MALL_ID = 'm_690'

In [3]:
# shop_infos
shop_infos = pd.read_csv(DATA_DIR + "训练数据-ccf_first_round_shop_info.csv")
shop_infos_1 = shop_infos[shop_infos.mall_id==MALL_ID]
shop_infos_1.index = range(shop_infos_1.shape[0])
print shop_infos_1.shape
shop_infos_1.head()

(137, 6)


,shop_id,category_id,longitude,latitude,price,mall_id
0,s_26,c_4,122.346736,31.833507,57,m_690
1,s_1587,c_1,122.347116,31.834759,60,m_690
2,s_1756,c_27,122.346919,31.833567,44,m_690
3,s_2258,c_38,122.346777,31.833647,50,m_690
4,s_2265,c_38,122.346113,31.834622,50,m_690


In [4]:
# user_shop_behavior
user_shop_behavior = pd.read_csv(DATA_DIR + "训练数据-ccf_first_round_user_shop_behavior.csv")
datasets_learn = pd.merge(user_shop_behavior,  shop_infos[['shop_id', 'mall_id']], on=['shop_id'])
datasets_learn_1 = datasets_learn[datasets_learn.mall_id==MALL_ID]
datasets_learn_1.insert(0, 'row_id', np.nan)
datasets_learn_1 = datasets_learn_1.loc[:, ['row_id', 'user_id', 'mall_id', 'time_stamp', 'longitude', 'latitude', 'wifi_infos', 'shop_id']]
print datasets_learn_1.shape
datasets_learn_1.head()

(26816, 8)


,row_id,user_id,mall_id,time_stamp,longitude,latitude,wifi_infos,shop_id
4535,NaN,u_3604,m_690,2017-08-15 09:40,122.346922,31.834588,b_33798074|-90|false;b_30210843|-66|false;b_50...,s_149642
4536,NaN,u_9950,m_690,2017-08-06 13:50,122.346942,31.834576,b_25145248|-55|false;b_17791227|-78|false;b_31...,s_149642
4537,NaN,u_34343,m_690,2017-08-06 19:20,122.346926,31.834624,b_22076003|-64|false;b_30210843|-74|false;b_50...,s_149642
4538,NaN,u_49943,m_690,2017-08-25 21:40,122.346950,31.834530,b_825631|-76|false;b_825628|-78|false;b_276007...,s_149642
4539,NaN,u_65787,m_690,2017-08-22 08:00,122.346947,31.834565,b_18516724|-72|false;b_21073988|-57|false;b_17...,s_149642


In [5]:
# evaluation public
datasets_predict = pd.read_csv(DATA_DIR + "AB榜测试集-evaluation_public.csv")
datasets_predict_1 = datasets_predict[datasets_predict.mall_id==MALL_ID].copy()
datasets_predict_1.loc[:, 'shop_id'] = np.nan
print datasets_predict_1.shape
datasets_predict_1.head()

(11808, 8)


,row_id,user_id,mall_id,time_stamp,longitude,latitude,wifi_infos,shop_id
237,118979,u_30176451,m_690,2017-09-07 11:30,122.346650,31.833528,b_39339645|-72|false;b_7446891|-51|false;b_233...,NaN
290,119032,u_30194619,m_690,2017-09-05 11:20,122.346477,31.834231,b_34360664|-61|false;b_34360668|-71|false;b_34...,NaN
326,119068,u_30205630,m_690,2017-09-07 17:00,122.346548,31.834173,b_34347400|-38|false;b_34360664|-59|false;b_34...,NaN
407,119149,u_30227745,m_690,2017-09-05 17:50,122.346467,31.834293,b_30227354|-53|false;b_33161184|-42|false;b_53...,NaN
413,119155,u_30229777,m_690,2017-09-03 11:30,122.346198,31.834007,b_7446881|-82|false;b_13587691|-63|false;b_343...,NaN


In [ ]:
## 学习部分

In [6]:
# datasets_learn_1和datasets_predict_1合并, 生成一个场景的数据集datasets_1
datasets_1 = pd.concat([datasets_learn_1, datasets_predict_1],  axis=0, ignore_index=True)
datasets_1.insert(0, 'sample_id', range(datasets_1.shape[0]))
print datasets_1.shape
datasets_1.head()

(38624, 9)


,sample_id,row_id,user_id,mall_id,time_stamp,longitude,latitude,wifi_infos,shop_id
0,0,NaN,u_3604,m_690,2017-08-15 09:40,122.346922,31.834588,b_33798074|-90|false;b_30210843|-66|false;b_50...,s_149642
1,1,NaN,u_9950,m_690,2017-08-06 13:50,122.346942,31.834576,b_25145248|-55|false;b_17791227|-78|false;b_31...,s_149642
2,2,NaN,u_34343,m_690,2017-08-06 19:20,122.346926,31.834624,b_22076003|-64|false;b_30210843|-74|false;b_50...,s_149642
3,3,NaN,u_49943,m_690,2017-08-25 21:40,122.346950,31.834530,b_825631|-76|false;b_825628|-78|false;b_276007...,s_149642
4,4,NaN,u_65787,m_690,2017-08-22 08:00,122.346947,31.834565,b_18516724|-72|false;b_21073988|-57|false;b_17...,s_149642


In [7]:
gc.collect()

218

In [10]:
## 构造训练特征
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# 扩展经纬度特征（可选）
for k in range(shop_infos_1.shape[0]):
    long_diff_name = 'lg_diff_' + shop_infos_1.loc[k, 'shop_id']
    long_diff = 1000 * (datasets_1.loc[:, 'longitude']  - shop_infos_1.loc[k, 'longitude'])
    scaler1 = MinMaxScaler()
    datasets_1.loc[:, long_diff_name] = scaler1.fit_transform(long_diff.values.reshape(-1,1))
    lati_diff_name = 'lt_diff_' + shop_infos_1.loc[k, 'shop_id']
    lati_diff = 1000 * (datasets_1.loc[:, 'latitude']  - shop_infos_1.loc[k, 'latitude'])
    scaler2 = MinMaxScaler()
    datasets_1.loc[:, lati_diff_name] = scaler2.fit_transform(lati_diff.values.reshape(-1,1))

datasets_1.loc[:, 'lg_org'] = MinMaxScaler().fit_transform(datasets_1.loc[:, 'longitude'].values.reshape(-1,1))
datasets_1.loc[:, 'lt_org']  = MinMaxScaler().fit_transform(datasets_1.loc[:, 'latitude'].values.reshape(-1,1))

print datasets_1.shape
datasets_1.head()

In [8]:
# 扩展时间特征（可选）
import time
datasets_1.loc[:, 'wday'] = datasets_1.time_stamp.apply(lambda t: time.strptime(t, '%Y-%m-%d %H:%M').tm_wday)
datasets_1.loc[:, 'hour'] = datasets_1.time_stamp.apply(lambda t: time.strptime(t, '%Y-%m-%d %H:%M').tm_hour)
datasets_1.loc[:, 'min'] = datasets_1.time_stamp.apply(lambda t: time.strptime(t, '%Y-%m-%d %H:%M').tm_min)
datasets_1.loc[:, 'time_str'] = map(lambda *xlist: "-".join([str(x) for x in xlist]),   
                                    datasets_1.loc[:, 'wday'], 
                                    datasets_1.loc[:, 'hour'], 
                                    datasets_1.loc[:, 'min'])

timestr_dicts = datasets_1.loc[:, 'time_str'].value_counts().to_dict()
timestr_cutted_dicts = { }
timestr_ignore_size = 0
for k, v in timestr_dicts.iteritems():
    if v >= timestr_ignore_size:
        timestr_cutted_dicts[k] = k
    else:
        timestr_cutted_dicts[k] = -1
datasets_1.loc[:, 'time_str']  = datasets_1.loc[:, 'time_str'].map(timestr_cutted_dicts)

dd = pd.get_dummies(datasets_1.time_str)
dd.columns = ['time_str_' + str(d) for d in dd.columns]

datasets_1 = pd.concat([datasets_1, dd], axis=1, ignore_index=False)

del datasets_1['wday']
del datasets_1['hour']
del datasets_1['min']
del datasets_1['time_str']
del dd

print datasets_1.shape
datasets_1.head()

(38624, 748)


,sample_id,row_id,user_id,mall_id,time_stamp,longitude,latitude,wifi_infos,shop_id,time_str_0-0-10,...,time_str_6-8-20,time_str_6-8-30,time_str_6-8-40,time_str_6-8-50,time_str_6-9-0,time_str_6-9-10,time_str_6-9-20,time_str_6-9-30,time_str_6-9-40,time_str_6-9-50
0,0,NaN,u_3604,m_690,2017-08-15 09:40,122.346922,31.834588,b_33798074|-90|false;b_30210843|-66|false;b_50...,s_149642,0,...,0,0,0,0,0,0,0,0,0,0
1,1,NaN,u_9950,m_690,2017-08-06 13:50,122.346942,31.834576,b_25145248|-55|false;b_17791227|-78|false;b_31...,s_149642,0,...,0,0,0,0,0,0,0,0,0,0
2,2,NaN,u_34343,m_690,2017-08-06 19:20,122.346926,31.834624,b_22076003|-64|false;b_30210843|-74|false;b_50...,s_149642,0,...,0,0,0,0,0,0,0,0,0,0
3,3,NaN,u_49943,m_690,2017-08-25 21:40,122.346950,31.834530,b_825631|-76|false;b_825628|-78|false;b_276007...,s_149642,0,...,0,0,0,0,0,0,0,0,0,0
4,4,NaN,u_65787,m_690,2017-08-22 08:00,122.346947,31.834565,b_18516724|-72|false;b_21073988|-57|false;b_17...,s_149642,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# 扩展WIFI特征（可选）
from sklearn.preprocessing import MinMaxScaler
sample_id_list = []
wifi_id_list = []
signal_power_list = []
signal_flag_list = []
sample_size = datasets_1.shape[0]
for i in range(sample_size):
    wifi_info = datasets_1.wifi_infos[i].split(';')
    sample_id = i
    for w in wifi_info:
        w_values = w.split('|')
        wifi_id_list.append(w_values[0])
        signal_power_list.append(float(w_values[1]))
        signal_flag_list.append(w_values[2])
        sample_id_list.append(sample_id)

wifi_feat = pd.DataFrame({'sample_id': sample_id_list,
                         'signal_power': signal_power_list,
                         'signal_flag': signal_flag_list,
                         'wifi_id': wifi_id_list},
                        columns=['sample_id', 'wifi_id', 'signal_power', 'signal_flag'])
wifi_feat['signal_flag'] = wifi_feat.signal_flag.apply(lambda x: 1 if x == 'true' else 0)
wifi_feat['signal_power'] = MinMaxScaler().fit_transform(wifi_feat['signal_power'].values.reshape(-1,1))

wifi_flag = pd.pivot_table(wifi_feat, index='sample_id', columns='wifi_id', values='signal_flag').fillna(0)
wifi_power = pd.pivot_table(wifi_feat, index='sample_id', columns='wifi_id', values='signal_power').fillna(0)
del wifi_feat

datasets_1 = pd.concat([datasets_1, wifi_power], axis=1, ignore_index=False)
del wifi_power
del wifi_flag

print datasets_1.shape
datasets_1.head()

(38624, 18338)


,sample_id,row_id,user_id,mall_id,time_stamp,longitude,latitude,wifi_infos,shop_id,time_str_0-0-10,...,b_993839,b_9947673,b_9949422,b_9952527,b_9968000,b_9989476,b_999413,b_999414,b_9996625,b_9996651
0,0,NaN,u_3604,m_690,2017-08-15 09:40,122.346922,31.834588,b_33798074|-90|false;b_30210843|-66|false;b_50...,s_149642,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,NaN,u_9950,m_690,2017-08-06 13:50,122.346942,31.834576,b_25145248|-55|false;b_17791227|-78|false;b_31...,s_149642,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,NaN,u_34343,m_690,2017-08-06 19:20,122.346926,31.834624,b_22076003|-64|false;b_30210843|-74|false;b_50...,s_149642,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,NaN,u_49943,m_690,2017-08-25 21:40,122.346950,31.834530,b_825631|-76|false;b_825628|-78|false;b_276007...,s_149642,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,NaN,u_65787,m_690,2017-08-22 08:00,122.346947,31.834565,b_18516724|-72|false;b_21073988|-57|false;b_17...,s_149642,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
gc.collect()

726

In [13]:
# 切分训练集, 预测集
from sklearn.utils import shuffle
datasets_sample_in = shuffle(datasets_1[datasets_1.shop_id.notnull()])
datasets_sample_out =  shuffle(datasets_1[datasets_1.shop_id.isnull()])

In [14]:
# 使用逻辑回归训练模型
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.model_selection import train_test_split
from pandas.core.series import Series

X_train, X_test, y_train, y_test  = train_test_split(datasets_sample_in.iloc[:,9:], 
                                                     datasets_sample_in.iloc[:,8], 
                                                     test_size=0.4, 
                                                     random_state=1)

lr_model = LogisticRegression(C=1.0, solver='lbfgs', multi_class='multinomial')
lr_model.fit(X_train,y_train)
p_train = Series(lr_model.predict(X_train))
lr_model.score(X_train, y_train)

0.93884020137982471

In [ ]:
0.93884020137982471
0.93828081297781096

In [ ]:
0.97673489765351973

In [ ]:
0.97423864203694455
0.94578132800798798

In [ ]:
0.97623564653020467
0.93030454318522215

In [ ]:
0.97553669495756368
0.92521218172740893